In [ ]:
import json
import torch
import torch.nn as nn
import commentjson
import numpy as np
import os
import gc
import logging  # 파이썬 기본 로깅
from transformers import EarlyStoppingCallback
from transformers import (
    AutoTokenizer,
    AutoModel,
    PreTrainedModel,
    TrainingArguments,
    Trainer
)
# transformers의 로깅을 별도 이름으로 임포트
from transformers import logging as transformers_logging
from datasets import Dataset, Features, Value, Sequence
from sklearn.metrics import accuracy_score, f1_score as sklearn_f1_score
from seqeval.metrics import f1_score, classification_report
from torchcrf import CRF
from tqdm import tqdm
import random
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# --- 0. 기본 설정 ---
MODEL_NAME = "klue/roberta-base"  # 모델 크기 증가 (base -> large)
MAX_LEN = 256  # 시퀀스 길이 증가
EPOCHS = 10    # 에폭 수 증가
BATCH_SIZE = 16  # 배치 크기 증가
LEARNING_RATE = 3e-5  # 학습률 최적화
WARMUP_RATIO = 0.1
WEIGHT_DECAY = 0.01
INTENT_WEIGHT = 0.4  # Intent 태스크 가중치 조정
NER_WEIGHT = 0.6     # NER 태스크 가중치 조정 (더 어려운 태스크에 가중치 부여)
# 모델 저장 경로 설정
INTEGRATED_MODEL_DIR = "./models/integrated_nlu_improved"
INTEGRATED_LABEL_PATH = os.path.join(INTEGRATED_MODEL_DIR, "nlu_labels.jsonc")
# Seed 설정 (재현성)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
# 로깅 설정
transformers_logging.set_verbosity_info()  # transformers 로깅 설정
logger = logging.getLogger(__name__)  # 파이썬 기본 로깅 설정
# --- 나머지 코드는 동일 ---
# --- 1. 데이터 로드 함수 ---
def load_intent_data():
    print("인텐트 데이터 로드 중...")
    with open('intent_label_list.jsonc', 'r', encoding='utf-8') as f:
        intent_label_list = commentjson.load(f)

    # 레이블 매핑 정보 상세 출력 (디버깅용)
    print(f"인텐트 레이블 목록: {intent_label_list}")
    intent_label_to_id = {label: i for i, label in enumerate(intent_label_list)}
    intent_id_to_label = {i: label for i, label in enumerate(intent_label_list)}
    
    print("레이블 매핑 정보:")
    for label, idx in intent_label_to_id.items():
        print(f"  '{label}' -> {idx}")

    with open('intent_data.jsonc', 'r', encoding='utf-8') as f:
        intent_data = commentjson.load(f)

    # 변환 전 원본 레이블 분포 확인
    original_labels = {}
    for item in intent_data:
        # 가능한 모든 레이블 필드 확인
        for key in ["intent_label", "intent", "label"]:
            if key in item:
                label_value = item[key]
                if label_value not in original_labels:
                    original_labels[label_value] = 0
                original_labels[label_value] += 1
                break
    
    print("\n변환 전 원본 레이블 분포:")
    for label, count in sorted(original_labels.items()):
        print(f"  '{label}': {count}개")

    # 인텐트 데이터의 레이블 형식 확인 및 통일 (개선된 버전)
    conversion_count = {"success": 0, "default": 0}
    for item in intent_data:
        # 원본 레이블 값 저장 (디버깅용)
        original_label = None
        converted_label = None
        
        # 1. intent_label 필드 처리
        if "intent_label" in item:
            original_label = item["intent_label"]
            # 이미 숫자인 경우
            if isinstance(item["intent_label"], int):
                # 유효 범위 확인
                if 0 <= item["intent_label"] < len(intent_label_list):
                    converted_label = item["intent_label"]
                    conversion_count["success"] += 1
                else:
                    # 범위 벗어나면 0으로 기본 설정
                    print(f"경고: 범위 벗어난 레이블 값 ({item['intent_label']}) - 0으로 설정")
                    item["intent_label"] = 0
                    converted_label = 0
                    conversion_count["default"] += 1
            # 문자열 레이블이면 ID로 변환
            elif isinstance(item["intent_label"], str):
                if item["intent_label"] in intent_label_to_id:
                    item["intent_label"] = intent_label_to_id[item["intent_label"]]
                    converted_label = item["intent_label"]
                    conversion_count["success"] += 1
                else:
                    print(f"경고: 알 수 없는 레이블 '{item['intent_label']}' - 0으로 설정")
                    item["intent_label"] = 0
                    converted_label = 0
                    conversion_count["default"] += 1
        
        # 2. intent 필드 처리
        elif "intent" in item and isinstance(item["intent"], str):
            original_label = item["intent"]
            if item["intent"] in intent_label_to_id:
                item["intent_label"] = intent_label_to_id[item["intent"]]
                converted_label = item["intent_label"]
                conversion_count["success"] += 1
            else:
                print(f"경고: 알 수 없는 intent '{item['intent']}' - 0으로 설정")
                item["intent_label"] = 0
                converted_label = 0
                conversion_count["default"] += 1
        
        # 3. label 필드 처리
        elif "label" in item and isinstance(item["label"], str):
            original_label = item["label"]
            if item["label"] in intent_label_to_id:
                item["intent_label"] = intent_label_to_id[item["label"]]
                converted_label = item["intent_label"]
                conversion_count["success"] += 1
            else:
                print(f"경고: 알 수 없는 label '{item['label']}' - 0으로 설정")
                item["intent_label"] = 0
                converted_label = 0
                conversion_count["default"] += 1
        
        # 4. 레이블이 없는 경우
        else:
            print(f"경고: 레이블 필드 없음 - 0으로 설정")
            item["intent_label"] = 0
            original_label = "없음"
            converted_label = 0
            conversion_count["default"] += 1
            
        # 샘플 출력 (처음 10개 항목만)
        if conversion_count["success"] + conversion_count["default"] <= 10:
            print(f"변환: '{original_label}' -> {converted_label} (텍스트: '{item.get('text', '')[0:30]}...')")

    print(f"\n레이블 변환 결과: 성공={conversion_count['success']}개, 기본값 사용={conversion_count['default']}개")

    # 레이블 분포 검증 추가 (개선된 버전)
    label_counts = {}
    for item in intent_data:
        label = item.get("intent_label", 0)
        if label not in label_counts:
            label_counts[label] = 0
        label_counts[label] += 1
    
    print("\n변환 후 인텐트 레이블 분포:")
    for label_id, count in sorted(label_counts.items()):
        label_name = intent_id_to_label.get(label_id, "알 수 없음")
        print(f"  - 레이블 {label_id} ({label_name}): {count}개")
    
    # 문제가 있는 경우 경고
    if len(label_counts) == 1 and 0 in label_counts:
        print("\n경고: 모든 레이블이 0번으로 설정되었습니다! intent_label_list와 데이터의 레이블이 일치하는지 확인하세요.")
    elif label_counts.get(0, 0) > 0.9 * sum(label_counts.values()):
        print(f"\n경고: 대부분의 레이블({label_counts.get(0, 0)}개, {label_counts.get(0, 0)/sum(label_counts.values())*100:.1f}%)이 0번으로 설정되었습니다!")
    
    return intent_data, intent_label_list, intent_label_to_id, intent_id_to_label

def validate_intent_labels(train_dataset, intent_id_to_label):
    """학습 데이터의 레이블 분포를 검증하고 불균형을 확인합니다."""
    label_counts = {}
    
    for item in train_dataset:
        label = item["intent_label"]
        if label not in label_counts:
            label_counts[label] = 0
        label_counts[label] += 1
    
    print("\n훈련 데이터 인텐트 레이블 분포:")
    total = sum(label_counts.values())
    for label_id, count in sorted(label_counts.items()):
        label_name = intent_id_to_label.get(label_id, "알 수 없음")
        percentage = (count / total) * 100
        print(f"  - 레이블 {label_id} ({label_name}): {count}개 ({percentage:.1f}%)")
    
    # 불균형 확인
    if len(label_counts) == 1:
        print("\n심각한 경고: 단일 레이블만 존재! 모델 학습이 제대로 되지 않을 것입니다.")
        return False
    
    max_label = max(label_counts, key=label_counts.get)
    max_count = label_counts[max_label]
    
    if max_count > 0.9 * total:
        print(f"\n경고: 레이블 {max_label}이 전체 데이터의 {max_count/total*100:.1f}%를 차지합니다. 균형 조정이 필요합니다.")
        return False
        
    return True

# --- 2. 통합 NLU 모델 정의 (Intent + NER) - 개선 ---
class ImprovedRobertaForJointIntentAndNER(PreTrainedModel):
    def __init__(self, config, intent_label_to_id, ner_label_to_id):
        super().__init__(config)
        self.num_intent_labels = len(intent_label_to_id)
        self.num_ner_labels = len(ner_label_to_id)

        # 기본 모델 로드
        self.roberta = AutoModel.from_pretrained(MODEL_NAME, config=config)

        # 인텐트 분류기 강화 
        self.intent_dropout = nn.Dropout(0.3)  # 드롭아웃 증가
        self.intent_hidden = nn.Linear(config.hidden_size, config.hidden_size)
        self.intent_activation = nn.GELU()
        self.intent_classifier = nn.Linear(config.hidden_size, self.num_intent_labels)

        # NER을 위한 헤드
        self.ner_dropout = nn.Dropout(0.3)
        self.ner_lstm = nn.LSTM(
            config.hidden_size,
            config.hidden_size // 2,
            num_layers=2,
            bidirectional=True,
            batch_first=True,
            dropout=0.2
        )
        self.ner_classifier = nn.Linear(config.hidden_size, self.num_ner_labels)

        # CRF 레이어 (NER용)
        self.crf = CRF(self.num_ner_labels, batch_first=True)

        # 손실 가중치 조정 (문제해결: NER 손실이 너무 커서 인텐트 비중 증가)
        self.intent_loss_weight = 0.8
        self.ner_loss_weight = 0.2

        # 가중치 초기화
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        intent_labels=None,
        ner_labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        # RoBERTa 인코더 실행
        outputs = self.roberta(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        sequence_output = outputs[0]  # 모든 토큰의 임베딩
        pooled_output = sequence_output[:, 0, :]  # [CLS] 토큰 임베딩 (Intent 용)

        # Intent 분류 (개선된 분류기)
        intent_output = self.intent_dropout(pooled_output)
        intent_hidden = self.intent_hidden(intent_output)
        intent_hidden = self.intent_activation(intent_hidden)
        intent_logits = self.intent_classifier(intent_hidden)
        
        # 디버깅: 인텐트 로짓 출력
        if intent_logits is not None and intent_labels is not None:
            probs = F.softmax(intent_logits, dim=-1)
            # 모든 클래스의 평균 확률 출력
            class_probs = probs.mean(dim=0)
            
            # 클래스별 최대 확률과 최소 확률 출력
            max_prob, max_idx = torch.max(class_probs, dim=0)
            min_prob, min_idx = torch.min(class_probs, dim=0)
            print(f"클래스별 확률 - 최대: 클래스 {max_idx.item()} ({max_prob.item():.4f}), 최소: 클래스 {min_idx.item()} ({min_prob.item():.4f})")
            
            # 무작위 샘플 5개에 대한 예측 확인 (인텐트별 확률 상위 3개 표시)
            if intent_logits.size(0) > 5:
                sample_indices = torch.randint(0, intent_logits.size(0), (5,))
                for i, idx in enumerate(sample_indices):
                    pred = torch.argmax(intent_logits[idx]).item()
                    true_label = intent_labels[idx].item()
                    
                    # 상위 3개 확률 출력
                    top_probs, top_indices = torch.topk(probs[idx], 3)
                    top_probs_str = ", ".join([f"클래스 {idx.item()}({prob.item():.4f})" for idx, prob in zip(top_indices, top_probs)])
                    
                    print(f"샘플 {i}: 예측={pred}, 실제={true_label}, 상위확률=[{top_probs_str}]")

        # NER 분류 (BiLSTM + CRF)
        ner_output = self.ner_dropout(sequence_output)
        ner_lstm_output, _ = self.ner_lstm(ner_output)
        ner_logits = self.ner_classifier(ner_lstm_output)

        # 손실 계산
        total_loss = None
        intent_loss = None
        ner_loss = None

        if intent_labels is not None:
            # 클래스 가중치 계산 - 희소 클래스에 더 높은 가중치 부여
            if hasattr(self, 'class_weights') and self.class_weights is not None:
                # 클래스 가중치가 있으면 사용
                intent_loss_fct = nn.CrossEntropyLoss(weight=self.class_weights)
            else:
                # 기본 손실 함수
                intent_loss_fct = nn.CrossEntropyLoss(label_smoothing=0.05)  # 레이블 스무딩 조정
            
            # 레이블 범위 검증 (오류 방지)
            if torch.max(intent_labels) >= self.num_intent_labels:
                print(f"경고: 인텐트 레이블 범위 초과 - 최대값: {torch.max(intent_labels).item()}, 허용 최대: {self.num_intent_labels-1}")
                # 범위를 벗어난 레이블 수정
                intent_labels = torch.clamp(intent_labels, 0, self.num_intent_labels-1)
            
            intent_loss = intent_loss_fct(intent_logits.view(-1, self.num_intent_labels), intent_labels.view(-1))

        if ner_labels is not None:
            # NER 손실 계산 (CRF)
            mask = attention_mask.bool()

            # -100을 유효한 레이블 인덱스로 변환 (CRF용)
            label_mask = ner_labels != -100
            crf_labels = ner_labels.clone()
            crf_labels[~label_mask] = 0

            # CRF 손실 계산
            ner_loss = -self.crf(ner_logits, crf_labels, mask=mask).mean()

            # NER 손실이 너무 크면 스케일링 (안정성 확보)
            if ner_loss > 100:
                ner_loss = ner_loss / (ner_loss.item() / 100)

            if torch.isnan(ner_loss) or torch.isinf(ner_loss):
                print("NER 손실이 NaN 또는 Inf입니다!")
                ner_loss = torch.tensor(0.1, device=ner_logits.device)

        # 총 손실 계산 (두 작업의 가중치 합)
        if intent_loss is not None and ner_loss is not None:
            # NER 손실 스케일링 (너무 큰 경우)
            if ner_loss.item() > 100 * intent_loss.item():
                scale_factor = ner_loss.item() / (100 * intent_loss.item())
                ner_loss = ner_loss / scale_factor
                print(f"NER 손실이 너무 커서 스케일링합니다: {scale_factor:.2f}배 감소")
            
            total_loss = self.intent_loss_weight * intent_loss + self.ner_loss_weight * ner_loss

        # 예측 단계
        intent_predictions = None
        ner_predictions = None

        if intent_labels is None:
            intent_predictions = torch.argmax(intent_logits, dim=1)

        if ner_labels is None:
            if attention_mask is not None:
                mask = attention_mask.bool()
                best_ner_tags_list = self.crf.decode(ner_logits, mask=mask)

                # 리스트를 텐서로 변환
                ner_predictions = torch.zeros_like(input_ids)
                for i, tags in enumerate(best_ner_tags_list):
                    ner_predictions[i, :len(tags)] = torch.tensor(tags, device=ner_predictions.device)
            else:
                best_ner_tags_list = self.crf.decode(ner_logits)
                ner_predictions = torch.tensor(best_ner_tags_list, device=ner_logits.device)

        return {
            "loss": total_loss,
            "intent_loss": intent_loss,
            "ner_loss": ner_loss,
            "intent_logits": intent_logits,
            "ner_logits": ner_logits,
            "intent_predictions": intent_predictions,
            "ner_predictions": ner_predictions
        }
    
def calculate_class_weights(intent_data, intent_label_to_id):
    """클래스별 가중치 계산 - 희소 클래스에 더 높은 가중치"""
    class_counts = {}
    for item in intent_data:
        # 다양한 레이블 키 시도
        intent_label = None
        possible_keys = ["intent_label", "intent", "label"]
        
        for key in possible_keys:
            if key in item:
                label_value = item[key]
                # 정수 타입이든 문자열이든 적절히 처리
                if isinstance(label_value, (int, float)):
                    intent_label = int(label_value)
                    break
                elif isinstance(label_value, str):
                    # 문자열이 숫자처럼 보이면 숫자로 변환 시도
                    if label_value.isdigit():
                        intent_label = int(label_value)
                    else:
                        # 문자열 레이블을 ID로 변환
                        intent_label = intent_label_to_id.get(label_value, 0)
                    break
        
        # 레이블을 찾지 못한 경우 기본값 사용
        if intent_label is None:
            intent_label = 0
            
        # 레이블 유효성 검사
        if intent_label >= len(intent_label_to_id):
            print(f"경고: 유효하지 않은 인텐트 레이블({intent_label}) - 기본값 0으로 설정")
            intent_label = 0
            
        if intent_label not in class_counts:
            class_counts[intent_label] = 0
        class_counts[intent_label] += 1
    
    # 클래스 분포 로깅 (디버깅용)
    print("\n인텐트 클래스 분포:")
    for intent_id, count in sorted(class_counts.items()):
        print(f"  - 클래스 {intent_id}: {count}개")
    
    # 가장 많은 클래스와 적은 클래스 찾기
    max_count = max(class_counts.values()) if class_counts else 1
    
    # 클래스별 가중치 계산 (적을수록 높은 가중치)
    class_weights = []
    for i in range(len(intent_label_to_id)):
        count = class_counts.get(i, 1)
        weight = max_count / (count + 1)  # 0으로 나누는 것 방지
        class_weights.append(weight)
    
    print("클래스별 가중치:", class_weights)
    return torch.tensor(class_weights, dtype=torch.float)

# --- 3. 데이터 전처리 및 통합 ---
def preprocess_and_merge_data():
    print("\n" + "="*50)
    print("데이터 전처리 및 통합 시작")
    print("="*50)

    # 3.1 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    # 3.2 Intent 데이터 로드 (개선된 함수 사용)
    intent_data, intent_label_list, intent_label_to_id, intent_id_to_label = load_intent_data()
    print(f"Intent 레이블 ({len(intent_label_list)}개): {intent_label_list}")

    # 3.3 NER 데이터 로드
    ner_data = load_ner_data()

    # 3.4 NER 레이블 정의 (BIO 형식)
    entity_types = set()
    for item in ner_data:
        for _, _, label in item["entities"]:
            entity_types.add(label)
    entity_types = sorted(list(entity_types))

    ner_labels = ["O"]  # Outside tag
    for entity_type in entity_types:
        ner_labels.extend([f"B-{entity_type}", f"I-{entity_type}"])

    ner_label_to_id = {label: i for i, label in enumerate(ner_labels)}
    ner_id_to_label = {i: label for label, i in ner_label_to_id.items()}
    print(f"NER 레이블 ({len(ner_labels)}개): {ner_labels}")

    # 3.5 데이터 통합 및 증강
    integrated_data = []

    # Intent 데이터 전처리 및 추가
    print("\n--- Intent 데이터 전처리 ---")

    # 각 인텐트 클래스별 데이터 수 파악 (개선: 레이블 일관성 확보)
    intent_class_counts = {}
    for item in intent_data:
        intent_label = item.get("intent_label", 0)  # 이미 정수로 변환됨
        
        if intent_label not in intent_class_counts:
            intent_class_counts[intent_label] = 0
        intent_class_counts[intent_label] += 1

    max_count = max(intent_class_counts.values())
    oversampling_rates = {label: max(1, min(5, int(max_count / count * 2))) 
                        for label, count in intent_class_counts.items()}
    
    print("인텐트 클래스별 데이터 수:")
    for label, count in intent_class_counts.items():
        label_name = intent_id_to_label.get(label, "알 수 없음")
        print(f"  - {label} ({label_name}): {count}개")

    print("클래스별 오버샘플링 비율:")
    for label, rate in oversampling_rates.items():
        label_name = intent_id_to_label.get(label, "알 수 없음")
        print(f"  - {label} ({label_name}): {rate}배")

    # Intent 데이터 추가 (개선: 레이블 처리 일관성)
    for idx, item in enumerate(tqdm(intent_data, desc="인텐트 데이터 처리")):
        try:
            # 텍스트 필드 확인
            if "text" not in item:
                continue

            text = item["text"]
            
            # 이미 정수로 변환된 레이블 사용
            intent_label = item.get("intent_label", 0)
            
            # 레이블 유효성 검사
            if intent_label >= len(intent_label_list) or intent_label < 0:
                print(f"경고: 유효하지 않은 인텐트 레이블({intent_label}) - 기본값 0으로 설정")
                intent_label = 0

            # 토큰화
            tokenized = tokenizer(
                text,
                return_offsets_mapping=True,
                add_special_tokens=True,
                truncation=True,
                max_length=MAX_LEN,
                padding='max_length'
            )
            input_ids = tokenized["input_ids"]
            attention_mask = tokenized["attention_mask"]
            offset_mapping = tokenized["offset_mapping"]

            # NER 레이블은 모두 무시 (-100)
            ner_token_labels = [-100] * len(input_ids)

            if idx < 3:  # 샘플 출력
                print(f"\n[Intent 데이터 {idx}] '{text}' -> 의도: {intent_id_to_label.get(intent_label, '알 수 없음')} ({intent_label})")

            integrated_data.append({
                "text": text,
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "intent_label": intent_label,
                "ner_labels": ner_token_labels
            })

            # 데이터 증강: 소량의 노이즈 추가 (랜덤 마스킹)
            if intent_label in oversampling_rates:
                for i in range(oversampling_rates[intent_label] - 1):  # 원본은 이미 추가했으므로 -1
                    # 약간의 변형 추가 (토큰 마스킹)
                    noised_input_ids = input_ids.copy()
                    for j in range(1, len(noised_input_ids)-1):
                        if random.random() < 0.1:  # 10% 확률로 마스킹
                            noised_input_ids[j] = tokenizer.mask_token_id
                    
                    integrated_data.append({
                        "text": text + f" (augmented {i+1})",
                        "input_ids": noised_input_ids,
                        "attention_mask": attention_mask,
                        "intent_label": intent_label,
                        "ner_labels": ner_token_labels
                    })

        except Exception as e:
            print(f"Intent 데이터 처리 중 오류 발생 (항목 {idx}): {e}")
            continue

    # NER 데이터 전처리 및 추가
    print("\n--- NER 데이터 전처리 ---")

    # 엔티티 유형별 개수 파악
    entity_type_counts = {}
    for item in ner_data:
        for _, _, entity_type in item["entities"]:
            if entity_type not in entity_type_counts:
                entity_type_counts[entity_type] = 0
            entity_type_counts[entity_type] += 1

    print("엔티티 유형별 개수:")
    for entity_type, count in entity_type_counts.items():
        print(f"  - {entity_type}: {count}개")

    # Intent 간 균형을 위한 인텐트 분포 생성
    intent_counts = {i: 0 for i in range(len(intent_label_list))}
    for item in integrated_data:
        intent_label = item.get("intent_label")
        if intent_label is not None:
            intent_counts[intent_label] += 1

    # 가장 많은 수의 인텐트 ID 찾기
    max_intent_count = max(intent_counts.values())
    min_intent_count = min(intent_counts.values())
    intent_weights = {i: max_intent_count / (count + 1) for i, count in intent_counts.items()}
    entity_samples_weight = 3
    # NER 데이터에 인텐트 분포 적용
    for idx, item in enumerate(tqdm(ner_data, desc="NER 데이터 처리")):
        text = item["text"]
        entities = item["entities"]

        

        # 적절한 인텐트 할당 (매핑 또는 무작위)
        # 의도적으로 부족한 인텐트 클래스에 가중치 부여하여 할당
        weighted_intent_ids = []
        for i, weight in intent_weights.items():
            weighted_intent_ids.extend([i] * int(weight * 10))

        intent_label = random.choice(weighted_intent_ids)

        # 텍스트 기반 인텐트 힌트 (텍스트에 특정 키워드가 있으면 관련 인텐트로 설정)
        # 예: "책, 도서, 대출" -> 도서 관련 인텐트
        # 이 부분은 프로젝트에 맞게 구체적으로 구현 필요

        # 1. 문자 단위 BIO 태깅
        char_labels = ["O"] * len(text)

        # 2. 엔티티에 따라 BIO 태그 할당
        for start_char, end_char, entity_type in entities:
            if start_char < 0:
                start_char = 0
            if end_char > len(text):
                end_char = len(text)

            if start_char < end_char and start_char < len(text):
                for i in range(start_char, end_char):
                    if i == start_char:
                        char_labels[i] = f"B-{entity_type}"
                    else:
                        char_labels[i] = f"I-{entity_type}"

        # 3. 토큰화 및 토큰-문자 정렬
        tokenized = tokenizer(
            text,
            return_offsets_mapping=True,
            add_special_tokens=True,
            truncation=True,
            max_length=MAX_LEN,
            padding='max_length'  # 패딩 추가
        )
        input_ids = tokenized["input_ids"]
        attention_mask = tokenized["attention_mask"]
        offset_mapping = tokenized["offset_mapping"]
        tokens = tokenizer.convert_ids_to_tokens(input_ids)

        # 4. 토큰별 레이블 할당 (개선된 BIO 태깅)
        ner_token_labels = []
        prev_entity_type = None
        prev_token_end = None

        for i, (start, end) in enumerate(offset_mapping):
            if start == end:  # 특수 토큰
                ner_token_labels.append(-100)
                continue
                
            # 현재 토큰이 어떤 엔티티에 속하는지 확인
            entity_type = None
            is_begin = False
            
            for ent_start, ent_end, ent_type in entities:
                # 토큰이 엔티티 범위에 포함되는지 확인
                if start >= ent_start and end <= ent_end:
                    entity_type = ent_type
                    # 시작 토큰인지 확인 (정확한 시작 부분 또는 처음 겹치는 부분)
                    is_begin = (start == ent_start or 
                            (prev_token_end is not None and prev_token_end <= ent_start < start))
                    break
            
            if entity_type:
                tag = f"B-{entity_type}" if is_begin else f"I-{entity_type}"
                ner_token_labels.append(ner_label_to_id[tag])
            else:
                ner_token_labels.append(ner_label_to_id["O"])
            
            prev_token_end = end

        # 레이블 길이 확인 및 조정
        if len(ner_token_labels) < len(input_ids):
            ner_token_labels.extend([-100] * (len(input_ids) - len(ner_token_labels)))
        elif len(ner_token_labels) > len(input_ids):
            ner_token_labels = ner_token_labels[:len(input_ids)]

        if idx < 3:  # 샘플 출력
            print(f"\n[NER 데이터 {idx}] '{text}' -> 엔티티: {entities}")
            print(f"  인텐트: {intent_id_to_label.get(intent_label, '알 수 없음')}")
            print(f"  토큰화: {tokens[:10]}... (총 {len(tokens)}개)")
            print(f"  NER 레이블: {[ner_id_to_label.get(l, 'IGN') for l in ner_token_labels[:10]]}... (총 {len(ner_token_labels)}개)")

        integrated_data.append({
            "text": text,
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "intent_label": intent_label,
            "ner_labels": ner_token_labels
        })

        # 데이터 증강: 엔티티가 많은 데이터는 중복해서 추가
        num_entities = len(entities)
        for i in range(min(num_entities * entity_samples_weight, 5)):  # 최대 5번 반복
            integrated_data.append({
                "text": f"{text} (duplicate {i+1})",
                "input_ids": input_ids,
                "attention_mask": attention_mask,
                "intent_label": intent_label,
                "ner_labels": ner_token_labels
            })

        print(f"\n통합 데이터셋 크기: {len(integrated_data)}")

    # 3.6 데이터셋 생성
    integrated_features = Features({
        'text': Value('string'),
        'input_ids': Sequence(Value('int32'), length=MAX_LEN),
        'attention_mask': Sequence(Value('int32'), length=MAX_LEN),
        'intent_label': Value('int32'),
        'ner_labels': Sequence(Value('int32'), length=MAX_LEN)
    })

    integrated_dataset = Dataset.from_list(integrated_data, features=integrated_features)

    # 학습/검증/테스트 데이터셋 분리
    train_val_test_datasets = integrated_dataset.train_test_split(test_size=0.3, seed=SEED)
    train_dataset = train_val_test_datasets["train"]

    # 테스트 데이터를 검증과 테스트로 분리
    test_valid_datasets = train_val_test_datasets["test"].train_test_split(test_size=0.5, seed=SEED)
    valid_dataset = test_valid_datasets["train"]
    test_dataset = test_valid_datasets["test"]

    print(f"훈련 데이터 크기: {len(train_dataset)}")
    print(f"검증 데이터 크기: {len(valid_dataset)}")
    print(f"테스트 데이터 크기: {len(test_dataset)}")

    return train_dataset, valid_dataset, test_dataset, intent_label_to_id, intent_id_to_label, ner_label_to_id, ner_id_to_label

# --- 4. 데이터 콜레이터 정의 ---
class ImprovedJointNLUDataCollator:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.pad_token_id = tokenizer.pad_token_id

    def __call__(self, features):
        # 입력 시퀀스 최대 길이 계산
        max_length = max(len(feature["input_ids"]) for feature in features)
        max_length = min(max_length, MAX_LEN)  # 최대 길이 제한

        # 배치 준비
        batch = {
            "input_ids": [],
            "attention_mask": [],
            "intent_labels": [],
            "ner_labels": []
        }

        for feature in features:
            # 패딩 적용
            input_ids = feature["input_ids"]
            attention_mask = feature["attention_mask"]
            ner_labels = feature["ner_labels"]

            # 길이 확인 및 조정
            if len(input_ids) > max_length:
                input_ids = input_ids[:max_length]
                attention_mask = attention_mask[:max_length]
                ner_labels = ner_labels[:max_length]

            padding_length = max_length - len(input_ids)

            # 입력 패딩
            if padding_length > 0:
                input_ids = input_ids + [self.pad_token_id] * padding_length
                attention_mask = attention_mask + [0] * padding_length
                ner_labels = ner_labels + [-100] * padding_length  # -100은 손실 계산에서 무시됨

            # 배치에 추가
            batch["input_ids"].append(input_ids)
            batch["attention_mask"].append(attention_mask)
            batch["intent_labels"].append(feature.get("intent_label", 0))
            batch["ner_labels"].append(ner_labels)

        # 텐서로 변환
        batch["input_ids"] = torch.tensor(batch["input_ids"], dtype=torch.long)
        batch["attention_mask"] = torch.tensor(batch["attention_mask"], dtype=torch.long)
        batch["intent_labels"] = torch.tensor(batch["intent_labels"], dtype=torch.long)
        batch["ner_labels"] = torch.tensor(batch["ner_labels"], dtype=torch.long)

        return batch

# --- 5. 개선된 커스텀 트레이너 정의 (통합 NLU용) ---
class ImprovedJointNLUTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.best_metrics = {'joint_score': 0}

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # 안정적인 손실 계산을 위한 예외 처리 추가
        try:
            outputs = model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                intent_labels=inputs["intent_labels"],
                ner_labels=inputs["ner_labels"]
            )

            loss = outputs["loss"]
            intent_loss = outputs["intent_loss"]
            ner_loss = outputs["ner_loss"]

            # 손실 값 자세히 로깅
            if self.state.global_step % 5 == 0:  # 5 스텝마다 출력
                print(f"Step {self.state.global_step} - Total: {loss.item():.4f}, Intent: {intent_loss.item() if intent_loss is not None else 0:.4f}, NER: {ner_loss.item() if ner_loss is not None else 0:.4f}")

            # 손실 값 검증 (NaN 방지)
            if torch.isnan(loss) or torch.isinf(loss):
                logger.warning("발견된 NaN/Inf 손실값! 대체값 사용...")
                loss = torch.tensor(1.0, device=loss.device, requires_grad=True)

            # 자세한 로깅 (정기적으로)
            if self.state.global_step % 10 == 0:
                self.log({
                    "total_loss": loss.item(),
                    "intent_loss": intent_loss.item() if intent_loss is not None else 0,
                    "ner_loss": ner_loss.item() if ner_loss is not None else 0,
                    "lr": self.optimizer.param_groups[0]['lr']  # 현재 학습률 로깅
                })

            return (loss, outputs) if return_outputs else loss

        except Exception as e:
            logger.error(f"손실 계산 중 오류 발생: {e}")
            # 오류 시 기본 손실 반환
            loss = torch.tensor(1.0, device=inputs["input_ids"].device, requires_grad=True)
            return (loss, None) if return_outputs else loss

    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        with torch.no_grad():
            # 입력 데이터 검증
            for k, v in inputs.items():
                if torch.isnan(v).any() or torch.isinf(v).any():
                    logger.warning(f"입력 데이터 문제 발견 ({k})!")
                    inputs[k] = torch.nan_to_num(v)  # NaN/Inf 값 대체

            outputs = model(
                input_ids=inputs["input_ids"],
                attention_mask=inputs["attention_mask"],
                intent_labels=inputs["intent_labels"],
                ner_labels=inputs["ner_labels"]
            )

            loss = outputs["loss"]

            if prediction_loss_only:
                return (loss, None, None)

            # 예측 및 레이블 추출
            intent_logits = outputs["intent_logits"]
            intent_predictions = torch.argmax(intent_logits, dim=1)
            intent_labels = inputs["intent_labels"]

            # CRF 기반 NER 예측 처리
            ner_logits = outputs["ner_logits"]
            attention_mask = inputs["attention_mask"].bool()
            ner_predictions = torch.zeros_like(inputs["input_ids"])

            try:
                # CRF 디코딩 (최적의 태그 시퀀스 찾기)
                best_ner_tags_list = model.crf.decode(ner_logits, mask=attention_mask)

                for i, tags in enumerate(best_ner_tags_list):
                    length = min(len(tags), ner_predictions.size(1))
                    ner_predictions[i, :length] = torch.tensor(tags[:length], device=ner_predictions.device)
            except Exception as e:
                logger.error(f"NER 예측 중 오류 발생: {e}")
                # 오류 시 모든 토큰을 'O'(Outside) 태그로 예측
                ner_predictions = torch.zeros_like(inputs["input_ids"])

            ner_labels = inputs["ner_labels"]

            # 반환값: (손실, 예측값 튜플, 레이블 튜플)
            return (loss, (intent_predictions, ner_predictions), (intent_labels, ner_labels))

    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        """향상된 평가 메서드: 점진적 평가 및 상세 메트릭스"""
        metrics = super().evaluate(eval_dataset, ignore_keys, metric_key_prefix)

        # 개선된 결과 저장 로직 (새로운 최고 성능 달성 시)
        current_score = metrics.get(f"{metric_key_prefix}_joint_score", 0)
        if current_score > self.best_metrics['joint_score']:
            self.best_metrics['joint_score'] = current_score
            self.best_metrics['step'] = self.state.global_step
            self.best_metrics['epoch'] = self.state.epoch

            # 상세 로깅
            logger.info(f"새로운 최고 성능 달성! Joint Score: {current_score:.4f} (Step: {self.state.global_step}, Epoch: {self.state.epoch:.2f})")

        return metrics

# --- 6. 개선된 평가 지표 계산 함수 ---
def compute_improved_joint_metrics(eval_preds, ner_label_list=None):
    (intent_preds, ner_preds), (intent_labels, ner_labels) = eval_preds

    # 인텐트 예측 분포 확인 (디버깅)
    intent_pred_counts = {}
    for pred in intent_preds:
        pred_id = pred.item()
        intent_name = intent_id_to_label.get(pred_id, "unknown")
        if intent_name not in intent_pred_counts:
            intent_pred_counts[intent_name] = 0
        intent_pred_counts[intent_name] += 1

    print("\n인텐트 예측 분포:")
    for intent_name, count in intent_pred_counts.items():
        print(f"  - {intent_name}: {count}개 ({count/len(intent_preds)*100:.2f}%)")

    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(intent_labels, intent_preds)
    
    # 혼동 행렬의 실제 크기 확인
    cm_size = cm.shape[0]
    # 표시할 클래스 수 결정 (지정한 값과 실제 크기 중 작은 값)
    top_classes = min(5, cm_size)
    
    print(f"\n상위 {top_classes}개 클래스에 대한 혼동 행렬:")
    for i in range(top_classes):
        row = ' '.join([f"{cm[i][j]:4d}" for j in range(top_classes)])
        print(f"  {i} [{intent_id_to_label.get(i, '?')}]: {row}")

    # Intent 평가 (F1 스코어 추가)
    intent_accuracy = accuracy_score(intent_labels, intent_preds)
    intent_f1_macro = sklearn_f1_score(intent_labels, intent_preds, average='macro')
    intent_f1_weighted = sklearn_f1_score(intent_labels, intent_preds, average='weighted')

    # NER 평가 준비
    true_ner_predictions = []
    true_ner_labels = []

    # attention_mask 생성 (ner_labels가 -100이 아닌 위치)
    attention_masks = []
    for label in ner_labels:
        mask = [l != -100 for l in label]
        attention_masks.append(mask)

    # attention_mask 포함하여 처리
    for prediction, label in zip(ner_preds, ner_labels):
        true_pred = []
        true_label = []
    
        for p, l in zip(prediction, label):
            if l != -100:  # 유효한 레이블만 평가
                p_id = p.item()
                l_id = l.item()
                
                # 레이블 ID가 유효한지 확인
                if p_id in ner_id_to_label and l_id in ner_id_to_label:
                    true_pred.append(ner_id_to_label[p_id])
                    true_label.append(ner_id_to_label[l_id]) 
        
        # 빈 시퀀스 문제 방지
        if len(true_pred) == 0:
            true_pred = ["O"]
            true_label = ["O"]
            
        true_ner_predictions.append(true_pred)
        true_ner_labels.append(true_label)

    # seqeval의 f1_score 계산 (더 상세한 메트릭스)
    try:
        ner_f1 = f1_score(true_ner_labels, true_ner_predictions)
        ner_report = classification_report(true_ner_labels, true_ner_predictions, digits=4, output_dict=True)

        # 개체 유형별 F1 점수 추출
        entity_metrics = {}
        for label, metrics in ner_report.items():
            if label not in ['micro avg', 'macro avg', 'weighted avg'] and isinstance(metrics, dict):
                entity_metrics[f"ner_f1_{label}"] = metrics.get('f1-score', 0)

        # 평가 보고서 출력 (텍스트 형식)
        print("\nNER Classification Report:\n", classification_report(true_ner_labels, true_ner_predictions, digits=4))

        # 통합 점수 계산 (가중 평균, NER에 더 높은 가중치)
        joint_score = INTENT_WEIGHT * intent_accuracy + NER_WEIGHT * ner_f1

        # 최종 메트릭스 반환 (상세한 지표 포함)
        metrics = {
            "intent_accuracy": intent_accuracy,
            "intent_f1_macro": intent_f1_macro,
            "intent_f1_weighted": intent_f1_weighted,
            "ner_f1": ner_f1,
            "joint_score": joint_score,
            **entity_metrics  # 개체 유형별 성능 추가
        }

        return metrics

    except Exception as e:
        logger.error(f"평가 지표 계산 오류: {e}")
        return {
            "intent_accuracy": intent_accuracy,
            "intent_f1_macro": intent_f1_macro,
            "intent_f1_weighted": intent_f1_weighted,
            "ner_f1": 0.0,
            "joint_score": intent_accuracy * INTENT_WEIGHT
        }

# --- 7. 학습률 스케줄러 개선 ---
def get_linear_schedule_with_warmup_and_decay(optimizer, num_warmup_steps, num_training_steps, min_lr_ratio=0.1):
    """학습률 워밍업 및 선형 감소 스케줄러"""

    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            # 워밍업 구간
            return float(current_step) / float(max(1, num_warmup_steps))

        # 워밍업 이후 선형 감소
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))
        return max(min_lr_ratio, 1.0 - progress)

    return torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

# --- 8. 개선된 통합 NLU 모델 훈련 함수 ---
def train_improved_integrated_nlu_model():
    print("\n" + "="*50)
    print("개선된 통합 NLU 모델 훈련 시작 (Intent + NER)")
    print("="*50)

    # 글로벌 변수 설정 (평가 지표용)
    global intent_id_to_label, ner_id_to_label, tokenizer

    # 8.1 데이터 전처리 및 통합
    train_dataset, valid_dataset, test_dataset, intent_label_to_id, intent_id_to_label, ner_label_to_id, ner_id_to_label = preprocess_and_merge_data()
    valid_labels = validate_intent_labels(train_dataset, intent_id_to_label)
    if not valid_labels:
        print("레이블 문제로 인해 학습을 중단합니다. 데이터와 레이블 매핑을 확인하세요.")
        return

    # 8.2 토크나이저 로드
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

    # 8.3 개선된 데이터 콜레이터 초기화
    data_collator = ImprovedJointNLUDataCollator(tokenizer)

    # 8.4 모델 설정
    config = AutoModel.from_pretrained(MODEL_NAME).config
    config.intent_label_to_id = intent_label_to_id
    config.ner_label_to_id = ner_label_to_id

    intent_data, _, _, _ = load_intent_data()
    class_weights = calculate_class_weights(intent_data, intent_label_to_id)

    # 8.5 개선된 통합 NLU 모델 초기화
    model = ImprovedRobertaForJointIntentAndNER(
        config,
        intent_label_to_id,
        ner_label_to_id
    )
    model.class_weights = class_weights.to(device)

    model = init_intent_classifier(model)

    # 8.6 개선된 훈련 설정
    training_args = TrainingArguments(
        output_dir="./results/improved_integrated_nlu",
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=2,
        logging_dir='./logs/improved_integrated_nlu',
        logging_steps=1,
        save_steps=50,
        eval_steps=25,
        eval_strategy="steps",
        load_best_model_at_end=True,
        save_total_limit=3,
        fp16=True,
        greater_is_better=True,
        metric_for_best_model="intent_accuracy",  # 인텐트 정확도를 기준으로 모델 선택
        weight_decay=0.01,
        learning_rate=5e-5,  # 학습률 증가
        warmup_ratio=0.1,
        lr_scheduler_type="linear",  # 단순한 스케줄러 사용
        report_to="none"
    )
    
    

    # 8.7 개선된 훈련 시작
    trainer = ImprovedJointNLUTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset,
        data_collator=data_collator,
        compute_metrics=compute_improved_joint_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    # 메모리 최적화
    gc.collect()
    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    # 훈련 실행
    print("모델 훈련 시작...")
    trainer.train()

    # 8.8 최종 평가
    print("\n--- 검증 세트 최종 평가 ---")
    eval_result = trainer.evaluate(valid_dataset)
    print(f"검증 세트 평가 결과: {eval_result}")

    print("\n--- 테스트 세트 최종 평가 ---")
    test_result = trainer.evaluate(test_dataset, metric_key_prefix="test")
    print(f"테스트 세트 평가 결과: {test_result}")

    # 8.9 모델 및 레이블 정보 저장
    print("\n모델 및 레이블 정보 저장 중...")
    os.makedirs(INTEGRATED_MODEL_DIR, exist_ok=True)

    # 메모리 최적화
    gc.collect()
    model.cpu()

    # 모델 저장
    torch.save(model.state_dict(), os.path.join(INTEGRATED_MODEL_DIR, "pytorch_model.bin"))

    # Config 정보 저장
    config_dict = config.to_dict()
    config_dict["model_type"] = "improved_roberta_joint_nlu"  # 개선된 모델 타입 표시
    with open(os.path.join(INTEGRATED_MODEL_DIR, "config.json"), "w", encoding="utf-8") as f:
        json.dump(config_dict, f, ensure_ascii=False, indent=4)

    # 토크나이저 저장
    tokenizer.save_pretrained(INTEGRATED_MODEL_DIR)

    # 레이블 매핑 저장
    with open(INTEGRATED_LABEL_PATH, 'w', encoding='utf-8') as f:
        json.dump({
            "intent_id2label": intent_id_to_label,
            "intent_label2id": intent_label_to_id,
            "ner_id2label": ner_id_to_label,
            "ner_label2id": ner_label_to_id,
            "training_metrics": {
                "validation": eval_result,
                "test": test_result,
                "best_metrics": trainer.best_metrics
            }
        }, f, ensure_ascii=False, indent=2)
    

    print(f"개선된 통합 NLU 모델 및 레이블 정보 저장 완료: {INTEGRATED_MODEL_DIR}")
    print("개선된 통합 NLU 모델 훈련 완료!")

def init_intent_classifier(model):
    """인텐트 분류기 가중치 특별 초기화 - 더 균형있게"""
    if hasattr(model, 'intent_classifier'):
        # Xavier 초기화 대신 작은 표준편차로 정규분포 초기화
        nn.init.normal_(model.intent_classifier.weight, mean=0.0, std=0.02)
        # 바이어스를 0으로 초기화하는 것이 아니라 작은 난수로 초기화
        nn.init.constant_(model.intent_classifier.bias, 0.1)
        print("인텐트 분류기 가중치 균형있게 초기화 완료")
    return model

# --- 9. 메인 실행 코드 ---
if __name__ == "__main__":
    try:
        # 시작 시간 기록
        import time
        start_time = time.time()

        # 개선된 훈련 함수 실행
        train_improved_integrated_nlu_model()

        # 총 실행 시간 출력
        total_time = time.time() - start_time
        hours, remainder = divmod(total_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        print(f"\n총 실행 시간: {int(hours)}시간 {int(minutes)}분 {seconds:.2f}초")

    except Exception as e:
        print(f"모델 훈련 중 오류 발생: {e}")
        import traceback
        traceback.print_exc()


개선된 통합 NLU 모델 훈련 시작 (Intent + NER)

데이터 전처리 및 통합 시작


loading file vocab.txt from cache at C:\Users\daehy\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\vocab.txt
loading file tokenizer.json from cache at C:\Users\daehy\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at C:\Users\daehy\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\special_tokens_map.json
loading file tokenizer_config.json from cache at C:\Users\daehy\.cache\huggingface\hub\models--klue--roberta-base\snapshots\02f94ba5e3fcb7e2a58a390b8639b0fac974a8da\tokenizer_config.json
loading file chat_template.jinja from cache at None


인텐트 데이터 로드 중...
인텐트 레이블 목록: ['search_book_title', 'search_book_author', 'search_book_location', 'check_book_availability', 'get_bestseller', 'get_new_releases', 'request_recommendation_genre', 'request_recommendation_mood', 'request_recommendation_topic', 'request_recommendation_similar', 'request_recommendation_reader', 'search_space_availability', 'reserve_space', 'get_space_info', 'check_space_reservation', 'cancel_space_reservation', 'search_program', 'apply_program', 'get_program_info', 'check_program_application', 'cancel_program_application', 'get_library_hours', 'inquire_service', 'manage_membership', 'check_loan_status', 'extend_loan', 'reserve_book', 'check_reservation_status', 'cancel_book_reservation', 'check_overdue_status', 'report_lost_item', 'greeting', 'gratitude', 'closing', 'affirmative', 'negative', 'abuse', 'clarification', 'out_of_scope', 'repeat']
레이블 매핑 정보:
  'search_book_title' -> 0
  'search_book_author' -> 1
  'search_book_location' -> 2
  'check_book_availab

인텐트 데이터 처리:  18%|█▊        | 241/1371 [00:00<00:00, 2404.76it/s]


[Intent 데이터 0] '나루토 책 있어?' -> 의도: search_book_title (0)

[Intent 데이터 1] '식물학자의 숲속 일기라는 책 있나요?' -> 의도: search_book_title (0)

[Intent 데이터 2] '사랑의 기술 책 찾아주세요.' -> 의도: search_book_title (0)


...

클래스별 확률 - 최대: 클래스 16 (0.0297), 최소: 클래스 22 (0.0219)
샘플 0: 예측=16, 실제=0, 상위확률=[클래스 16(0.0314), 클래스 1(0.0309), 클래스 4(0.0299)]
샘플 1: 예측=16, 실제=0, 상위확률=[클래스 16(0.0314), 클래스 1(0.0309), 클래스 4(0.0299)]
샘플 2: 예측=16, 실제=0, 상위확률=[클래스 16(0.0326), 클래스 7(0.0314), 클래스 1(0.0303)]
샘플 3: 예측=16, 실제=0, 상위확률=[클래스 16(0.0324), 클래스 1(0.0312), 클래스 10(0.0311)]
샘플 4: 예측=1, 실제=0, 상위확률=[클래스 1(0.0314), 클래스 16(0.0313), 클래스 4(0.0300)]
클래스별 확률 - 최대: 클래스 16 (0.0289), 최소: 클래스 18 (0.0221)
샘플 0: 예측=7, 실제=0, 상위확률=[클래스 7(0.0311), 클래스 1(0.0309), 클래스 16(0.0304)]
샘플 1: 예측=7, 실제=0, 상위확률=[클래스 7(0.0311), 클래스 1(0.0309), 클래스 16(0.0304)]
샘플 2: 예측=11, 실제=0, 상위확률=[클래스 11(0.0270), 클래스 16(0.0265), 클래스 3(0.0261)]
샘플 3: 예측=1, 실제=0, 상위확률=[클래스 1(0.0315), 클래스 16(0.0308), 클래스 26(0.0294)]
샘플 4: 예측=13, 실제=0, 상위확률=[클래스 13(0.0268), 클래스 0(0.0268), 클래스 14(0.0265)]
클래스별 확률 - 최대: 클래스 16 (0.0293), 최소: 클래스 22 (0.0219)
샘플 0: 예측=0, 실제=0, 상위확률=[클래스 0(0.0279), 클래스 30(0.0271), 클래스 13(0.0270)]
샘플 1: 예측=1, 실제=0, 상위확률=[클래스 1(0.0317), 클래스 16(0.0316), 클래스 0(0.029

The following columns in the evaluation set don't have a corresponding argument in `ImprovedRobertaForJointIntentAndNER.forward` and have been ignored: text, intent_label. If text, intent_label are not expected by `ImprovedRobertaForJointIntentAndNER.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 687
  Batch size = 16


클래스별 확률 - 최대: 클래스 0 (0.0972), 최소: 클래스 8 (0.0194)
샘플 0: 예측=0, 실제=0, 상위확률=[클래스 0(0.0970), 클래스 21(0.0291), 클래스 7(0.0281)]
샘플 1: 예측=0, 실제=0, 상위확률=[클래스 0(0.0974), 클래스 21(0.0293), 클래스 1(0.0280)]
샘플 2: 예측=0, 실제=0, 상위확률=[클래스 0(0.0949), 클래스 21(0.0293), 클래스 7(0.0288)]
샘플 3: 예측=0, 실제=0, 상위확률=[클래스 0(0.0975), 클래스 21(0.0293), 클래스 1(0.0283)]
샘플 4: 예측=0, 실제=0, 상위확률=[클래스 0(0.0974), 클래스 21(0.0284), 클래스 7(0.0279)]
클래스별 확률 - 최대: 클래스 0 (0.0972), 최소: 클래스 9 (0.0193)
샘플 0: 예측=0, 실제=0, 상위확률=[클래스 0(0.0989), 클래스 21(0.0300), 클래스 1(0.0284)]
샘플 1: 예측=0, 실제=0, 상위확률=[클래스 0(0.0953), 클래스 21(0.0298), 클래스 1(0.0287)]
샘플 2: 예측=0, 실제=0, 상위확률=[클래스 0(0.0928), 클래스 21(0.0289), 클래스 1(0.0285)]
샘플 3: 예측=0, 실제=0, 상위확률=[클래스 0(0.0982), 클래스 21(0.0288), 클래스 1(0.0281)]
샘플 4: 예측=0, 실제=0, 상위확률=[클래스 0(0.0995), 클래스 21(0.0292), 클래스 7(0.0280)]
클래스별 확률 - 최대: 클래스 0 (0.0967), 최소: 클래스 8 (0.0195)
샘플 0: 예측=0, 실제=0, 상위확률=[클래스 0(0.0969), 클래스 21(0.0293), 클래스 1(0.0283)]
샘플 1: 예측=0, 실제=0, 상위확률=[클래스 0(0.0972), 클래스 21(0.0293), 클래스 7(0.0285)]
샘플 2: 예측=0, 실

c:\Users\daehy\Desktop\ai\venv\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(
c:\Users\daehy\Desktop\ai\venv\Lib\site-packages\seqeval\metrics\v1.py:159: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
c:\Users\daehy\Desktop\ai\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\daehy\Desktop\ai\venv\Lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true sample


인텐트 예측 분포:
  - search_book_title: 687개 (100.00%)

상위 1개 클래스에 대한 혼동 행렬:
  0 [search_book_title]:  687
클래스별 확률 - 최대: 클래스 0 (0.0948), 최소: 클래스 36 (0.0193)
샘플 0: 예측=0, 실제=0, 상위확률=[클래스 0(0.0842), 클래스 1(0.0303), 클래스 10(0.0292)]
샘플 1: 예측=0, 실제=0, 상위확률=[클래스 0(0.1039), 클래스 21(0.0296), 클래스 7(0.0288)]
샘플 2: 예측=0, 실제=0, 상위확률=[클래스 0(0.0976), 클래스 21(0.0323), 클래스 4(0.0293)]
샘플 3: 예측=0, 실제=0, 상위확률=[클래스 0(0.0883), 클래스 30(0.0297), 클래스 21(0.0273)]
샘플 4: 예측=0, 실제=0, 상위확률=[클래스 0(0.1160), 클래스 21(0.0330), 클래스 30(0.0289)]
Step 25 - Total: 1.8885, Intent: 2.3606, NER: -0.0000
클래스별 확률 - 최대: 클래스 0 (0.0910), 최소: 클래스 8 (0.0195)
샘플 0: 예측=0, 실제=0, 상위확률=[클래스 0(0.0798), 클래스 21(0.0288), 클래스 7(0.0274)]
샘플 1: 예측=0, 실제=0, 상위확률=[클래스 0(0.0929), 클래스 21(0.0288), 클래스 1(0.0286)]
샘플 2: 예측=0, 실제=0, 상위확률=[클래스 0(0.0968), 클래스 4(0.0324), 클래스 7(0.0291)]
샘플 3: 예측=0, 실제=0, 상위확률=[클래스 0(0.0929), 클래스 21(0.0288), 클래스 1(0.0286)]
샘플 4: 예측=0, 실제=0, 상위확률=[클래스 0(0.0968), 클래스 4(0.0324), 클래스 7(0.0291)]
Step 25 - Total: 1.9229, Intent: 2.4036, NER:

In [14]:
import commentjson

# 파일 로드
with open('intent_label_list.jsonc', 'r', encoding='utf-8') as f:
    intent_label_list = commentjson.load(f)

with open('intent_data.jsonc', 'r', encoding='utf-8') as f:
    intent_data = commentjson.load(f)

# 결과 출력
print(f"인텐트 레이블 목록: {intent_label_list}")
print(f"총 데이터 수: {len(intent_data)}")

# 레이블 분포 확인
label_counts = {}
for item in intent_data:
    # 레이블 값 가져오기
    label = item.get("intent_label", 0)  # 기본값 0
    
    if label not in label_counts:
        label_counts[label] = 0
    label_counts[label] += 1

# 결과 출력
print("\n레이블 분포:")
for label, count in sorted(label_counts.items()):
    print(f"레이블 {label}: {count}개")

인텐트 레이블 목록: ['search_book_title', 'search_book_author', 'search_book_location', 'check_book_availability', 'get_bestseller', 'get_new_releases', 'request_recommendation_genre', 'request_recommendation_mood', 'request_recommendation_topic', 'request_recommendation_similar', 'request_recommendation_reader', 'search_space_availability', 'reserve_space', 'get_space_info', 'check_space_reservation', 'cancel_space_reservation', 'search_program', 'apply_program', 'get_program_info', 'check_program_application', 'cancel_program_application', 'get_library_hours', 'inquire_service', 'manage_membership', 'check_loan_status', 'extend_loan', 'reserve_book', 'check_reservation_status', 'cancel_book_reservation', 'check_overdue_status', 'report_lost_item', 'greeting', 'gratitude', 'closing', 'affirmative', 'negative', 'abuse', 'clarification', 'out_of_scope', 'repeat']
총 데이터 수: 1371

레이블 분포:
레이블 0: 194개
레이블 1: 116개
레이블 2: 213개
레이블 3: 183개
레이블 4: 75개
레이블 5: 89개
레이블 6: 295개
레이블 7: 45개
레이블 8: 5개
레이블 9: 